# Prepare White-Matter Connecitvity Blueprints

This notebook loads individual blueprints and concatenates them into a single file.

**Inputs**

Indiviudal blueprints, of shape (41, 64984)<br>
`/scratch/users/robert.scholz2/hcp_conn_blueprints/{subj}.BP.LR.dscalar.ni`, 

**Outputs**

Single HCP Individual blueprints file<br>
`/scratch/users/robert.scholz2/dist_model/inputs/all_blueprints_BPsubset247.npy`

<br>

In [ ]:
import numpy as np
import nibabel as nib
import hcp_utils as hcp

In [2]:
#!tar -xf /scratch/users/robert.scholz2/hcp/hcpBlueprints.tar.gz -C /scratch/users/robert.scholz2/hcp_conn_blueprints 

# Load and then save all blueprints to a single file

In [3]:
subjs=np.loadtxt("data/subjs_hcp254_full_unrelated.txt").astype(int).astype(str);
len(subjs), subjs[:4]

(254, array(['100206', '100610', '101006', '101309'], dtype='<U21'))

In [ ]:
!ls /scratch/users/robert.scholz2/hcp_conn_blueprints | tail -n 4
bpc=nib.load("/scratch/users/robert.scholz2/hcp_conn_blueprints/996782.BP.LR.dscalar.nii")
bpd = bpc.get_fdata()
bpd.shape

In [ ]:
import os
#sbpa = np.zeros((len(bp_subjs), 1, 29696, 41))
bp_subjs = []
sbpa = []
bp_exists=[]
failed_subjs=[]

# blueprint file have bma:
#('CIFTI_STRUCTURE_CORTEX_LEFT', slice(0, 32492, None))
#('CIFTI_STRUCTURE_CORTEX_RIGHT', slice(32492, None, None))

for sn, subj in enumerate(subjs):
  print(sn, end=" "); 
  f = f"/scratch/users/robert.scholz2/hcp_conn_blueprints/{subj}.BP.LR.dscalar.nii"
  bp_exists.append(os.path.exists(f))
  if os.path.exists(f):
    lh = nib.load(f).get_fdata()[:, slice(0, 32492)] # (41, 64984) -- slice -> (41, 32492)
    bp_subjs.append(subj)
    sbpa.append(lh[:, hcp.vertex_info["grayl"]].T); # each item has shape: (29696, 41)
  else:
    failed_subjs.append(subj)

sbpa = np.array(sbpa)
n_subjs=len(bp_subjs)
print("Failed:", len(failed_subjs), failed_subjs)
print("\nFinal array: ", sbpa.shape)

In [ ]:
#path = "/home/users/robert.scholz2/bigdata/hcp_100L_subjs_sm12_BPsubset80.txt" #full_subj_path
path = "data/subjs_hcp254_full_unrelated_BPsubset247.txt" #full_subj_path
np.savetxt(path, bp_subjs, fmt="%s")

#sbpa = np.load(path).astype(np.float32)
bpdata = {subj : sbpa[i].astype(np.float32) for i, subj in enumerate(bp_subjs)}
np.save("/scratch/users/robert.scholz2/dist_model/inputs/all_blueprints_BPsubset247.npy", bpdata)
!ls -ash /scratch/users/robert.scholz2/dist_model/inputs/all_blueprints_BPsubset247.npy

# Information on the contained white matter tracts

In [ ]:
#xq = nib.load("/scratch/users/robert.scholz2/hcp_conn_blueprints/348545.BP.LR.dscalar.nii")
#wm_bundels_names = xq.header.get_axis(0).name
wm_bundels_names = ['af_l', 'af_r', 'ar_l', 'ar_r', 'atr_l', 'atr_r', 'cbd_l', 'cbd_r',
       'cbp_l', 'cbp_r', 'cbt_l', 'cbt_r', 'cst_l', 'cst_r', 'fa_l',
       'fa_r', 'fma', 'fmi', 'fx_l', 'fx_r', 'ilf_l', 'ilf_r', 'ifo_l',
       'ifo_r', 'mdlf_l', 'mdlf_r', 'or_l', 'or_r', 'str_l', 'str_r',
       'slf1_l', 'slf1_r', 'slf2_l', 'slf2_r', 'slf3_l', 'slf3_r', 'ac',
       'uf_l', 'uf_r', 'vof_l', 'vof_r']

# different order than:
#https://git.fmrib.ox.ac.uk/rmars/comparing-connectivity-blueprints/-/blob/master/structureList
# https://github.com/SPMIC-UoN/xtract

len(wm_bundels_names)

In [ ]:
np.savetxt("/scratch/users/robert.scholz2/dist_model/inputs/wm_blueprint_names.txt", wm_bundels_names, fmt="%s")

In [ ]:
wminfo = np.array([['Arcuate Fasciculus', 'AF', 'af_l af_r'], ['Acoustic Radiation', 'AR', 'ar_l ar_r'], ['Anterior Thalamic Radiation', 'ATR', 'atr_l atr_r'], ['Cingulum subsection : Dorsal', 'CBD', 'cbd_l cbd_r'], ['Cingulum subsection : Peri-genual', 'CBP', 'cbp_l cbp_r'], ['Cingulum subsection : Temporal', 'CBT', 'cbt_l cbt_r'], ['Corticospinal Tract', 'CST', 'cst_l cst_r'], ['Frontal Aslant', 'FA', 'fa_l fa_r'], ['Forceps Major', 'FMA', 'fma'], ['Forceps Minor', 'FMI', 'fmi'], ['Fornix', 'FX', 'fx_l fx_r'], ['Inferior Longitudinal Fasciculus', 'ILF', 'ilf_l ilf_r'], ['Inferior Fronto-Occipital Fasciculus', 'IFO', 'ifo_l ifo_r'], ['Middle Cerebellar Peduncle', 'MCP', 'mcp'], ['Middle Longitudinal Fasciculus', 'MdLF', 'mdlf_l mdlf_r'], ['Optic Radiation', 'OR', 'or_l or_r'], ['Superior Thalamic Radiation', 'STR', 'str_l str_r'], ['Superior Longitudinal Fasciculus 1', 'SLF1', 'slf1_l slf1_r'], ['Superior Longitudinal Fasciculus 2', 'SLF2', 'slf2_l slf2_r'], ['Superior Longitudinal Fasciculus 3', 'SLF3', 'slf3_l slf3_r'], ['Anterior Commissure', 'AC', 'ac'], ['Uncinate Fasciculus', 'UF', 'uf_l uf_r'], ['Vertical Occipital Fasciculus', 'VOF', 'vof_l vof_r']])
wminfo.shape

In [ ]:
bundle_info = []
for bn in wm_bundels_names:
  print(bn, end="\t")
  for x in wminfo: 
    if bn in x[-1]: 
      hemisp = "l" if bn.endswith("_l") else "";
      hemisp = "r" if bn.endswith("_r") else hemisp;
      bundle_info.append([bn, hemisp, x[0] ])
      print(hemisp+ x[0], "\t", )

bundle_info= np.array(bundle_info)
bundle_info.shape